## Basic setup and directory structure

In [ ]:
import os

BASE_DIR = '/home/rithomas'
# folder containing MIDI and/or MusicXML files. can have child folders.
RAW_DATA = os.path.join(BASE_DIR, 'data', 'clean_midi')
# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD = os.path.join(BASE_DIR, 'cache', 'performanceRNN', 'data', 'notesequences.tfrecord')
# Model config must be one of ['basic_improv', 'attention_improv', 'chord_pitches_improv']
PERFORMANCE_RNN_CONFIG = 'chord_pitches_improv'
# Model directory for storing weights and tensorboard stuff
PERFORMANCE_RNN_DIR = os.path.join(BASE_DIR, 'cache', 'performanceRNN', 'model')
# Dataset of Sequence Examples for the ImprovRNN Model
PERFORMANCE_RNN_DATASET = os.path.join(BASE_DIR, 'cache', 'performanceRNN')
# Training sequence examples
TRAINING_EXAMPLES = os.path.join(PERFORMANCE_RNN_DATASET, 'training_lead_sheets.tfrecord')
# Dir for testing outputs
OUTPUTS_PATH = os.path.join(PERFORMANCE_RNN_DATASET, 'outputs')
# Final bundled model
BUNDLE_DIR = os.path.join(PERFORMANCE_RNN_DIR, PERFORMANCE_RNN_CONFIG + '.mag')

if not os.path.exists(SEQUENCES_TFRECORD):
    !python3 "magenta/scripts/convert_dir_to_note_sequences.py" \
      --input_dir=$RAW_DATA \
      --output_file=$SEQUENCES_TFRECORD \
      --recursive

## Create PerformanceRNN Dataset

In [ ]:
# CONFIG=<one of 'performance', 'performance_with_dynamics', etc.>
performance_rnn_create_dataset \
--config=${CONFIG} \
--input=/tmp/notesequences.tfrecord \
--output_dir=/tmp/performance_rnn/sequence_examples \
--eval_ratio=0.10

## Train PerformanceRNN on Dataset 

In [ ]:
performance_rnn_train \
--config=${CONFIG} \
--run_dir=/tmp/performance_rnn/logdir/run1 \
--sequence_example_file=/tmp/performance_rnn/sequence_examples/training_performances.tfrecord

## Generate some outputs for testing

In [ ]:
performance_rnn_generate \
--run_dir=/tmp/performance_rnn/logdir/run1 \
--output_dir=/tmp/performance_rnn/generated \
--config=${CONFIG} \
--num_outputs=10 \
--num_steps=3000 \
--primer_melody="[60,62,64,65,67,69,71,72]"

## Save model if you like it!

In [ ]:
performance_rnn_generate \
  --config=${CONFIG} \
  --run_dir=/tmp/performance_rnn/logdir/run1 \
  --bundle_file=/tmp/performance_rnn.mag \
  --save_generator_bundle